In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.1 MB/s eta 0:00:00


# 슈퍼쏠 이상치 날짜별 토픽 모델링 #

In [21]:
import pandas as pd
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np

# tqdm.pandas() 추가: progress_apply를 사용할 수 있도록 설정
tqdm.pandas()

# 날짜 파일에서 날짜 읽기
with open('/content/drive/MyDrive/PML_PAPER/data/5_Anomaly_Dates_SuperSol.txt', 'r') as file:
    anomaly_dates = file.read().splitlines()

# 문자열을 datetime 객체로 변환
anomaly_dates = [datetime.strptime(date, "%Y-%m-%d") for date in anomaly_dates]

# Sentiment CSV 파일 읽기
df = pd.read_csv('/content/drive/MyDrive/PML_PAPER/data/4_Sentiment_SuperSol.csv')

# 'date' 컬럼을 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'])

stopwords = set([
    "신한", "쏠", "앱", "슈퍼쏠", "어플", "은행", "기능", "슈퍼", "금융", "대부분"
])

# 불용어 제거 함수
def remove_stopwords(tokens, stopwords):
    return [word for word in tokens if word not in stopwords and len(word) > 1]

# 3일 간의 데이터를 추출하는 함수
def get_3_day_data(date, df):
    start_date = date
    end_date = date + timedelta(days=2)
    subset_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    return subset_df
# 이전 3일 간의 데이터를 추출하는 함수
def get_previous_3_day_data(date, df):
    start_date = date - timedelta(days=3)
    end_date = date - timedelta(days=1)
    subset_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    return subset_df

# LDA 모델링 함수 수정
def lda_topic_modeling(text_data, sentiment_data, anomaly_date, df, num_topics=2):
    # 'nouns' 컬럼에서 NaN 값을 가진 행을 필터링
    text_data = text_data.dropna()

    # 불용어 및 한글자 명사 제거
    text_data = text_data.progress_apply(lambda x: remove_stopwords(eval(x), stopwords))

    # 필터링 후, 텍스트가 비어 있는지 확인
    if text_data.empty or text_data.apply(len).sum() == 0:
        print("No valid text data available for LDA modeling.")
        return

    # LDA 모델 학습을 위한 텍스트 준비
    texts = text_data.tolist()

    # Gensim Dictionary 생성
    dictionary = corpora.Dictionary(texts)

    # 코퍼스 생성
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Converting complex numbers (if any) to real numbers in the corpus
    corpus = [[(word_id, float(weight.real)) for word_id, weight in doc] for doc in corpus]

    # LDA 모델 학습
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=20)

    # LDA 모델의 토픽 출력
    topics = lda_model.print_topics(num_words=10)
    for topic in topics:
        print(topic)

    # Assign topics to each review
    topic_assignments = [max(lda_model[doc], key=lambda x: x[1])[0] for doc in corpus]

    # Create a DataFrame with topic assignments
    topic_counts = pd.Series(topic_assignments).value_counts()

    # 평균 센티먼트 스코어 계산
    avg_sentiment = sentiment_data.mean()

    # 이전 3일 평균 센티먼트 스코어 계산
    prev_data_3days = get_previous_3_day_data(anomaly_date, df)
    prev_sentiment_data = prev_data_3days['sentiment_score']
    prev_avg_sentiment = prev_sentiment_data.mean() if not prev_sentiment_data.empty else 0

    # 이전 3일의 기간 계산
    prev_start_date = (anomaly_date - timedelta(days=3)).strftime('%Y-%m-%d')
    prev_end_date = (anomaly_date - timedelta(days=1)).strftime('%Y-%m-%d')

    # 증감율 계산
    if prev_avg_sentiment != 0:
        sentiment_change = ((avg_sentiment - prev_avg_sentiment) / prev_avg_sentiment) * 100
    else:
        sentiment_change = float('inf')  # 이전 센티먼트가 0인 경우 무한대로 처리

    # 기간 출력 추가
    print(f"\nPeriod: {anomaly_date.strftime('%Y-%m-%d')} to {(anomaly_date + timedelta(days=2)).strftime('%Y-%m-%d')}")
    print(f"Total reviews from {anomaly_date.strftime('%Y-%m-%d')} to {(anomaly_date + timedelta(days=2)).strftime('%Y-%m-%d')}: {len(text_data)} reviews")
    print(f"Topic-wise review count:")
    for topic_num, count in topic_counts.items():
        print(f"Topic {topic_num}: {count} reviews")

    print(f"Average sentiment score: {avg_sentiment:.4f}")

    # 이전 3일 기간 및 평균 센티먼트 출력
    print(f"Previous 3 days' period: {prev_start_date} to {prev_end_date}")
    print(f"Previous 3 days' average sentiment score: {prev_avg_sentiment:.4f}")

    print(f"Sentiment change: {sentiment_change:.2f}%")

    # Visualize the topics
    pyLDAvis.enable_notebook()
    vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

    # Save the visualization to an HTML file
    file_name = f"/content/drive/MyDrive/PML_PAPER/data/6_SuperSol_LDA_{anomaly_date.strftime('%Y-%m-%d')}.html"
    pyLDAvis.save_html(vis_data, file_name)
    print(f"Saved LDA visualization to {file_name}\n\n\n\n")


# 모든 날짜에 대해 반복하여 결과 처리
for anomaly_date in anomaly_dates:
    # 해당 날짜에 대해 3일 간의 데이터를 가져오기
    data_3days = get_3_day_data(anomaly_date, df)

    # sentiment 컬럼이 있다고 가정하여, sentiment 데이터도 추출
    sentiment_data = data_3days['sentiment_score']

    # 3일 간의 리뷰 텍스트에 대해 LDA 토픽 모델링 시각화 및 리뷰 수 출력
    lda_topic_modeling(data_3days['nouns'], sentiment_data, anomaly_date, df)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 24/24 [00:00<00:00, 9999.33it/s]


(0, '0.058*"인증" + 0.037*"기존" + 0.037*"결제" + 0.037*"추가" + 0.037*"사람" + 0.030*"사용" + 0.026*"계좌" + 0.026*"휴대폰" + 0.026*"조회" + 0.020*"카드"')
(1, '0.088*"카드" + 0.043*"증권" + 0.042*"편리" + 0.031*"디자인" + 0.027*"통합" + 0.027*"사용" + 0.019*"이체" + 0.019*"주식" + 0.019*"내역" + 0.019*"기간"')

Period: 2024-02-19 to 2024-02-21
Total reviews from 2024-02-19 to 2024-02-21: 24 reviews
Topic-wise review count:
Topic 0: 14 reviews
Topic 1: 10 reviews
Average sentiment score: 0.6869
Previous 3 days' period: 2024-02-16 to 2024-02-18
Previous 3 days' average sentiment score: 0.2128
Sentiment change: 222.80%
Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_SuperSol_LDA_2024-02-19.html






100%|██████████| 59/59 [00:00<00:00, 20805.78it/s]


(0, '0.104*"사용" + 0.104*"편리" + 0.066*"포인트" + 0.056*"설치" + 0.040*"기존" + 0.040*"결재" + 0.040*"조아요" + 0.024*"불편" + 0.024*"메시지" + 0.024*"별점"')
(1, '0.144*"최고" + 0.076*"이벤트" + 0.042*"시작" + 0.025*"응모" + 0.025*"계열" + 0.025*"한번" + 0.025*"일상" + 0.025*"퀴즈" + 0.025*"자산" + 0.025*"인원"')

Period: 2024-03-11 to 2024-03-13
Total reviews from 2024-03-11 to 2024-03-13: 59 reviews
Topic-wise review count:
Topic 0: 39 reviews
Topic 1: 20 reviews
Average sentiment score: 0.9121
Previous 3 days' period: 2024-03-08 to 2024-03-10
Previous 3 days' average sentiment score: 0.2979
Sentiment change: 206.19%
Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_SuperSol_LDA_2024-03-11.html






100%|██████████| 46/46 [00:00<00:00, 17749.58it/s]


...

(0, '0.161*"인증" + 0.097*"울트라" + 0.097*"비밀번호" + 0.097*"본인" + 0.097*"지문" + 0.097*"사용" + 0.097*"수단" + 0.032*"정상" + 0.032*"작동" + 0.032*"실행"')
(1, '0.097*"현상" + 0.097*"주기" + 0.097*"버튼" + 0.097*"아무것" + 0.097*"먹통" + 0.097*"실행" + 0.097*"작동" + 0.097*"정상" + 0.032*"인증" + 0.032*"수단"')

Period: 2024-09-16 to 2024-09-18
Total reviews from 2024-09-16 to 2024-09-18: 2 reviews
Topic-wise review count:
Topic 1: 1 reviews
Topic 0: 1 reviews
Average sentiment score: 0.3860
Previous 3 days' period: 2024-09-13 to 2024-09-15
Previous 3 days' average sentiment score: 0.5432
Sentiment change: -28.95%


Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_SuperSol_LDA_2024-09-16.html






100%|██████████| 7/7 [00:00<00:00, 4329.13it/s]

(0, '0.074*"조건" + 0.074*"통장" + 0.074*"처음" + 0.045*"급여" + 0.045*"이체" + 0.045*"개설" + 0.045*"대면" + 0.045*"방송" + 0.045*"시간" + 0.045*"이상"')
(1, '0.065*"답변" + 0.065*"변경" + 0.065*"해외" + 0.039*"제공" + 0.039*"개선" + 0.039*"소리" + 0.039*"메일" + 0.039*"설정" + 0.039*"의지" + 0.039*"테스트"')

Period: 2024-11-06 to 2024-11-08
Total reviews from 2024-11-06 to 2024-11-08: 7 reviews
Topic-wise review count:
Topic 1: 4 reviews
Topic 0: 3 reviews
Average sentiment score: 0.3092
Previous 3 days' period: 2024-11-03 to 2024-11-05
Previous 3 days' average sentiment score: 0.9749
Sentiment change: -68.28%


Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_SuperSol_LDA_2024-11-06.html






In [ ]:
from IPython.core.display import display, HTML

# HTML 파일 경로 (구글 드라이브에 저장된 경로 사용)
html_file_path = '/content/drive/MyDrive/PML_PAPER/data/6_SuperSol_LDA_2024-03-13.html'

# HTML 파일을 읽어 표시
display(HTML(html_file_path))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# KB스타 이상치 날짜별 토픽 모델링 #

In [20]:
import pandas as pd
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np

# tqdm.pandas() 추가: progress_apply를 사용할 수 있도록 설정
tqdm.pandas()

# 날짜 파일에서 날짜 읽기
with open('/content/drive/MyDrive/PML_PAPER/data/5_Anomaly_Dates_KBstar.txt', 'r') as file:
    anomaly_dates = file.read().splitlines()

# 문자열을 datetime 객체로 변환
anomaly_dates = [datetime.strptime(date, "%Y-%m-%d") for date in anomaly_dates]

# Sentiment CSV 파일 읽기
df = pd.read_csv('/content/drive/MyDrive/PML_PAPER/data/4_Sentiment_KBstar.csv')

# 'date' 컬럼을 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'])

stopwords = set([
    "국민", "앱", "KB", "어플", "스타뱅킹", "뱅킹", "사용", "스타", "은행"
])

# 이전 3일 간의 데이터를 추출하는 함수
def get_previous_3_day_data(date, df):
    start_date = date - timedelta(days=3)
    end_date = date - timedelta(days=1)
    subset_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    return subset_df

# 불용어 제거 함수
def remove_stopwords(tokens, stopwords):
    return [word for word in tokens if word not in stopwords and len(word) > 1]

# 필터링된 'nouns' 칼럼에서 4개 이상의 명사를 가진 데이터만 사용
def filter_nouns_by_length(df, min_length=4):
    # 'nouns' 컬럼이 리스트 형태여야 하고, 그 길이가 min_length 이상인 경우에만 필터링
    return df[df['nouns'].apply(lambda x: len(eval(x)) >= min_length)]

# LDA 모델링 함수 수정
def lda_topic_modeling(text_data, sentiment_data, anomaly_date, df, num_topics=3):
    # 'nouns' 컬럼에서 NaN 값을 가진 행을 필터링
    text_data = text_data.dropna()

    # 불용어 및 한글자 명사 제거
    text_data = text_data.progress_apply(lambda x: remove_stopwords(eval(x), stopwords))

    # 필터링 후, 텍스트가 비어 있는지 확인
    if text_data.empty or text_data.apply(len).sum() == 0:
        print("No valid text data available for LDA modeling.")
        return

    # LDA 모델 학습을 위한 텍스트 준비
    texts = text_data.tolist()

    # Gensim Dictionary 생성
    dictionary = corpora.Dictionary(texts)

    # 코퍼스 생성
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Converting complex numbers (if any) to real numbers in the corpus
    corpus = [[(word_id, float(weight.real)) for word_id, weight in doc] for doc in corpus]

    # LDA 모델 학습
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

    # LDA 모델의 토픽 출력
    topics = lda_model.print_topics(num_words=5)
    for topic in topics:
        print(topic)

    # Assign topics to each review
    topic_assignments = [max(lda_model[doc], key=lambda x: x[1])[0] for doc in corpus]

    # Create a DataFrame with topic assignments
    topic_counts = pd.Series(topic_assignments).value_counts()

    # 평균 센티먼트 스코어 계산
    avg_sentiment = sentiment_data.mean()

    # 이전 3일 평균 센티먼트 스코어 계산
    prev_data_3days = get_previous_3_day_data(anomaly_date, df)
    prev_sentiment_data = prev_data_3days['sentiment_score']
    prev_avg_sentiment = prev_sentiment_data.mean() if not prev_sentiment_data.empty else 0

    # 이전 3일의 기간 계산
    prev_start_date = (anomaly_date - timedelta(days=3)).strftime('%Y-%m-%d')
    prev_end_date = (anomaly_date - timedelta(days=1)).strftime('%Y-%m-%d')

    # 증감율 계산
    if prev_avg_sentiment != 0:
        sentiment_change = ((avg_sentiment - prev_avg_sentiment) / prev_avg_sentiment) * 100
    else:
        sentiment_change = float('inf')  # 이전 센티먼트가 0인 경우 무한대로 처리

    # 기간 출력 추가
    print(f"\nPeriod: {anomaly_date.strftime('%Y-%m-%d')} to {(anomaly_date + timedelta(days=2)).strftime('%Y-%m-%d')}")
    print(f"Total reviews from {anomaly_date.strftime('%Y-%m-%d')} to {(anomaly_date + timedelta(days=2)).strftime('%Y-%m-%d')}: {len(text_data)} reviews")
    print(f"Topic-wise review count:")
    for topic_num, count in topic_counts.items():
        print(f"Topic {topic_num}: {count} reviews")

    print(f"Average sentiment score: {avg_sentiment:.4f}")

    # 이전 3일 기간 및 평균 센티먼트 출력
    print(f"Previous 3 days' period: {prev_start_date} to {prev_end_date}")
    print(f"Previous 3 days' average sentiment score: {prev_avg_sentiment:.4f}")

    print(f"Sentiment change: {sentiment_change:.2f}%")

    # Visualize the topics
    pyLDAvis.enable_notebook()
    vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

    # Save the visualization to an HTML file
    file_name = f"/content/drive/MyDrive/PML_PAPER/data/6_KBstar_LDA_{anomaly_date.strftime('%Y-%m-%d')}.html"
    pyLDAvis.save_html(vis_data, file_name)
    print(f"Saved LDA visualization to {file_name}\n\n\n\n")


# 모든 날짜에 대해 반복하여 결과 처리
for anomaly_date in anomaly_dates:
    # 해당 날짜에 대해 3일 간의 데이터를 가져오기
    data_3days = get_previous_3_day_data(anomaly_date, df)

    # 필터링된 'nouns' 데이터를 사용하여 4개 이상의 명사를 가진 리뷰만 사용
    filtered_data_3days = filter_nouns_by_length(data_3days, min_length=4)

    # sentiment 컬럼이 있다고 가정하여, sentiment 데이터도 추출
    sentiment_data = filtered_data_3days['sentiment_score']

    # 3일 간의 리뷰 텍스트에 대해 LDA 토픽 모델링 시각화 및 리뷰 수 출력
    lda_topic_modeling(filtered_data_3days['nouns'], sentiment_data, anomaly_date, df)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 10/10 [00:00<00:00, 6225.77it/s]


(0, '0.175*"인증서" + 0.080*"로그인" + 0.056*"불편" + 0.056*"금융" + 0.032*"인식"')
(1, '0.085*"신분증" + 0.065*"인식" + 0.046*"오류" + 0.046*"적금" + 0.026*"카드"')
(2, '0.083*"상품" + 0.083*"가입" + 0.051*"편리" + 0.048*"적금" + 0.047*"걷기"')

Period: 2023-01-01 to 2023-01-03
Total reviews from 2023-01-01 to 2023-01-03: 10 reviews
Topic-wise review count:
Topic 1: 5 reviews
Topic 0: 3 reviews
Topic 2: 2 reviews
Average sentiment score: 0.5139
Previous 3 days' period: 2022-12-29 to 2022-12-31
Previous 3 days' average sentiment score: 0.8371
Sentiment change: -38.61%
Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_KBstar_LDA_2023-01-01.html






100%|██████████| 2/2 [00:00<00:00, 1572.96it/s]

(0, '0.114*"이체" + 0.114*"계좌" + 0.114*"휴먼" + 0.114*"잔액" + 0.114*"감사"')
(1, '0.130*"항목" + 0.091*"월급" + 0.091*"로그인" + 0.052*"작성" + 0.052*"호재"')
(2, '0.050*"감사" + 0.050*"잔액" + 0.050*"이체" + 0.050*"휴먼" + 0.050*"계좌"')

Period: 2023-01-10 to 2023-01-12
Total reviews from 2023-01-10 to 2023-01-12: 2 reviews
Topic-wise review count:
Topic 1: 1 reviews
Topic 0: 1 reviews
Average sentiment score: 0.5055
Previous 3 days' period: 2023-01-07 to 2023-01-09
Previous 3 days' average sentiment score: 0.9130
Sentiment change: -44.63%


Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_KBstar_LDA_2023-01-10.html






...

(0, '0.069*"신분증" + 0.041*"대면" + 0.039*"환전" + 0.039*"개설" + 0.039*"촬영"')
(1, '0.071*"모바일" + 0.050*"친절" + 0.050*"상담" + 0.049*"편리" + 0.028*"직원"')
(2, '0.056*"사용자" + 0.056*"암호" + 0.033*"이체" + 0.032*"계좌" + 0.032*"문제"')

Period: 2024-09-28 to 2024-09-30
Total reviews from 2024-09-28 to 2024-09-30: 10 reviews
Topic-wise review count:
Topic 1: 5 reviews
Topic 0: 3 reviews
Topic 2: 2 reviews
Average sentiment score: 0.6872
Previous 3 days' period: 2024-09-25 to 2024-09-27
Previous 3 days' average sentiment score: 0.8565
Sentiment change: -19.76%


Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_KBstar_LDA_2024-09-28.html






100%|██████████| 6/6 [00:00<00:00, 5067.62it/s]

(0, '0.049*"직원" + 0.049*"친절" + 0.049*"추종" + 0.049*"편리" + 0.049*"불허"')
(1, '0.099*"뎨좌" + 0.099*"대면" + 0.099*"개설" + 0.026*"결제" + 0.026*"카드"')
(2, '0.164*"결제" + 0.088*"진입" + 0.088*"카드" + 0.051*"불편" + 0.051*"인식"')

Period: 2024-10-10 to 2024-10-12
Total reviews from 2024-10-10 to 2024-10-12: 6 reviews
Topic-wise review count:
Topic 0: 3 reviews
Topic 2: 2 reviews
Topic 1: 1 reviews
Average sentiment score: 0.8207
Previous 3 days' period: 2024-10-07 to 2024-10-09
Previous 3 days' average sentiment score: 0.9096
Sentiment change: -9.78%


Saved LDA visualization to /content/drive/MyDrive/PML_PAPER/data/6_KBstar_LDA_2024-10-10.html






In [ ]:

from IPython.core.display import display, HTML

# HTML 파일 경로 (구글 드라이브에 저장된 경로 사용)
html_file_path = '/content/drive/MyDrive/PML_PAPER/data/6_KBstar_LDA_2023-09-01.html'

# HTML 파일을 읽어 표시
display(HTML(html_file_path))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
